# Part I. ETL Pipeline for Pre-Processing the Files

In [108]:
# importing Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

In [109]:
# checking current working directory
print(os.getcwd())

# getting current folder and subfolder event data
filepath = os.getcwd() + 

# creating a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# joining the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [110]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list,
for f in file_path_list:
    # read csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
         # extracting each data row one by one and append it
        for line in csvreader:
            full_data_rows_list.append(line) 
            
print("Total number of rows:", len(full_data_rows_list))

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

Total number of rows: 1008


In [111]:
# checking the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print("Number of rows:", sum(1 for line in f))

Number of rows: 855


# Part II. Complete the Apache Cassandra coding portion of your project. 

## The <font color=red>event_datafile_new.csv</font> file, located within the Workspace directory, contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is an example of what the denormalized data will appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [112]:
# This should make a connection to a Cassandra instance in the local machine (127.0.0.1)
from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, we need a session
session = cluster.connect()

#### Creating Keyspace

In [113]:
try: 
    session.execute("CREATE KEYSPACE IF NOT EXISTS sparkify WITH REPLICATION = {'class': 'SimpleStrategy', 'replication_factor': 1}")
except Exception as e: 
    print(e)

#### Setting Keyspace

In [114]:
try: 
    session.set_keyspace('sparkify')
except Exception as e: 
    print(e)

### To run the following queries, we will need to create distinct tables, because with Apache Cassandra you must model the database tables on the queries you wants to run.

## Goal Queries:

### 1. List of artists, song titles and songs' length in the music app history that were heard during sessionId = 338 and itemInSession  = 4


### 2. Name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10 and sessionid = 182
    

### 3. User names (first and last) in the music app history who listened to the song 'All Hands Against His Own'

------------------------------------------------------------------------------------------------------------

### Creating tables and running queries

### Query 1. Retrieve the artist, song title and song's length in the music app history that was heard during sessionId = 338 and itemInSession  = 4

In [115]:
query = "CREATE TABLE IF NOT EXISTS songs_by_sessions"
query = query + "(session_id int, item_session int, artist text, song_title text, song_length double, \
                    PRIMARY KEY (session_id, item_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [116]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO songs_by_sessions (session_id, item_session, artist, song_title, song_length)"
        query = query +  "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

In [117]:
query = "SELECT artist, song_title, song_length FROM songs_by_sessions WHERE session_id = 338 AND item_session = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, "-", row.song_title, ",", row.song_length, "seconds")

### Query 2: Retrieve the artists' names, the songs (sorted by itemInSession) and user name (first and last name) for userid = 10 and sessionid = 182

In [118]:
query2 = "CREATE TABLE IF NOT EXISTS songs_by_users_and_sessions"
query2 = query2 + "(user_id int, session_id int, item_session int, artist text, song_title text, user_firstname text, user_lastname text, \
                    PRIMARY KEY ((user_id, session_id), item_session))"
try:
    session.execute(query2)
except Exception as e:
    print(e)                    

In [119]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO songs_by_users_and_sessions (user_id, session_id, item_session, artist, song_title, user_firstname, user_lastname)"
        query = query +  "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

In [120]:
query = "SELECT artist, song_title, user_firstname, user_lastname FROM songs_by_user_and_session WHERE user_id = 10 AND session_id = 182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, "-", row.song_title, "(", row.user_firstname, row.user_lastname, ")")

Down To The Bone - Keep On Keepin' On ( Sylvie Cruz )
Three Drives - Greece 2000 ( Sylvie Cruz )
Sebastien Tellier - Kilometer ( Sylvie Cruz )
Lonnie Gordon - Catch You Baby (Steve Pitron & Max Sanna Radio Edit) ( Sylvie Cruz )


### Query 3: Retrieve user names (first and last) in the music app history who listened to the song 'All Hands Against His Own'

In [121]:
query3 = "CREATE TABLE IF NOT EXISTS users_by_song_titles "
query3 = query3 + "(song_title text, user_id int, user_firstname text, user_lastname text, \
                    PRIMARY KEY ((song_title), user_id))"
try:
    session.execute(query3)
except Exception as e:
    print(e)

In [122]:
# it is important to note that the order of columns in the INSERT statement must follow the primary key order
# for the same reason aforementioned in CREATE statement.
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO users_by_song_titles (song_title, user_id, user_firstname, user_lastname)"
        query = query +  "VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

In [123]:
query = "SELECT user_firstname, user_lastname FROM users_by_song_titles WHERE song_title = 'All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.user_firstname, row.user_lastname)

### Dropping the tables before closing out the sessions

In [124]:
drop1 = "DROP TABLE IF EXISTS songs_by_sessions"
drop2 = "DROP TABLE IF EXISTS songs_by_users_and_sessions"
drop3 = "DROP TABLE IF EXISTS users_by_song_titles"
try:
    rows = session.execute(drop1)
    rows = session.execute(drop1)
    rows = session.execute(drop1)
except Exception as e:
    print(e)

### Closing the session and cluster connection¶

In [125]:
session.shutdown()
cluster.shutdown()

#### Footnote: it is important to note that the order of columns in the CREATE statement must follow the primary key order, which means that the first columns in the CREATE clause consist of the partition key followed by all of the clustering columns in order. Only after that can we state other column names. This is required because Apache Cassandra determines which rows go to which nodes with the primary key for better efficiency when reading and writing.